In [25]:
import pandas as pd

import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.rcParams['timezone'] = 'Europe/Berlin'

import data_process_base
import importlib
importlib.reload(data_process_base)

try:
    import seaborn
    colors = seaborn.color_palette()
except:
    colors = ["b", "g", "r", "c", "y"]

In [26]:
# create random time series from import data
da = data_process_base.DataAction()
nc = data_process_base.net_calc()

keep_cols = ['DE_KN_residential1_grid_import',
             'DE_KN_residential2_grid_import',
             'utc_timestamp']
da.imp_procc("house_data.csv", keep_cols)

Number of data frame segments =  90
dfList created successfully.


In [32]:
def end_results(sgen_val, iter):
    """
    Final boss

    """
    # create df to store output
    col_names = ['line_1', 'line_2', 'line_3', 'line_4', 'line_5', 'line_6', 'line_7', 'line_8']
    end_vals = pd.DataFrame(columns=col_names)
    end_times = pd.DataFrame(dtype='str', columns=col_names)

    for i in range(0, iter):
        # iterative scramble
        night_merge = da.power_merge()

        # assign generator value at random times
        night_mw = da.sgen_comm(ts=night_merge, wind_length=60, sgen_val=sgen_val, parties=4)
        # night_mw = da.sgen_rand(ts=night_merge, sgen_val=sgen_val)

        nc.four_loads_branched_make(night_mw)
        nc.four_loads_branched_out("res_line", "loading_percent")
        nc.four_loads_branched_run()
        nc.four_loads_branched_read_loadpct()

        nc.end_vals_step(nc.ll, end_vals)
        nc.end_times_step(nc.ll, end_times)

    return end_vals, end_times


iter = 10
sgen_val = 0.01 # typical val: 0.010 MW = 10kW
end_vals, end_times = end_results(sgen_val, iter)

100%|██████████| 601/601 [00:01<00:00, 342.79it/s]


In [34]:
end_vals

,line_1,line_2,line_3,line_4,line_5,line_6,line_7,line_8
0,56.329185,30.830091,25.285763,8.522746,33.443170,8.506247,22.925660,8.523023
1,71.004068,66.544083,35.039399,8.508642,9.151695,35.234040,30.758303,8.508920


In [35]:
end_times

,line_1,line_2,line_3,line_4,line_5,line_6,line_7,line_8
0,18:02:00,18:02:00,18:01:00,21:01:00,18:11:00,22:02:00,18:02:00,21:01:00
1,18:57:00,18:57:00,19:11:00,22:40:00,03:43:00,19:00:00,19:11:00,22:40:00


In [28]:
nc.four_loads_branched_plot_linepct()

In [29]:
nc.load_graph(nc.net, time_step=45)